In [1]:
from korbash_lib import Puller, PlotDisplayer, Slider
import time
import numpy as np
import pandas as pd
import datetime
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
import scipy
from scipy import stats
import random
import os
output_notebook()
pl = Puller(simulate=True) #simulate=True
random.seed()

# инициализация запускать только один раз

Loading BokehJS ...

In [2]:
#pl.ms.motorM.MoveTo(30);

In [3]:
pl.ms.StopButton() # кнопки для остоновки моторчиков в нештатной ситуации

GridspecLayout(children=(Button(description='Stop All Motors', layout=Layout(grid_area='widget001'), style=But…

In [2]:
pl.ms.MoveToStart(zapas=10) # сдвиг моторчиков на начальные позиции для растяжки
pl.ms.motorM.MoveTo(17);
while pl.ms.IsInMotion():
    pass

In [ ]:
pl.tg.SetZeroWeight(15) # калибровка тензодатчика

In [2]:
x0=pl.SetH_avto() # измерение высоты молибдена

0.03130979185453874    -49.96869020814546    0.1155848690272698
45.93889610591334    -4.0611038940866635    0.009393925671232114
52.98730212640349    2.987302126403492    -0.006910065554789077
51.08076546574494    1.080765465744939    -0.002499968165804787
49.70620123116281    -0.2937987688371919    0.000679599406647731
49.40071750181839    -0.5992824981816085    0.0013862278313503764
49.88126611774739    -0.11873388225261294    0.0002746487885133799
50.030454223447585    0.03045422344758464    -7.044506097425128e-05
SetW finish value= 50.013825438915745
            time    tension     motorM
0   1.603716e+09  49.942967  15.761482
1   1.603716e+09  49.969120  15.748895
2   1.603716e+09  49.958991  15.727557
3   1.603716e+09  50.000110  15.697383
4   1.603716e+09  49.932839  15.658373
5   1.603716e+09  49.954758  15.609970
6   1.603716e+09  49.938205  15.553194
7   1.603716e+09  49.921576  15.487582
8   1.603716e+09  49.989452  15.414810
9   1.603716e+09  49.893837  15.335430
10  1.6037

podgon a= 6.9985364149181555   b= 49.31118217944893   x0= 14.957302960741417


data saved


In [5]:
pl.SetW(50, dw=0.1, tau=2,quiet=False); # натяжка волокна

49.976898587064774    -0.02310141293522605    5.338310285841051e-05
SetW finish value= 49.954012373386554


In [13]:
pl.ms.motorM.MoveTo(pl.ms.motorM.position_max-2); # опускание молибдена вниз перед разогревом

In [14]:
pl.tg.SetZeroWeight(5,50) # калибровка тензодатчика после разогрева молибдена

67.93195673056367

In [3]:
x0=11.6
pl.ms.x0=x0

In [8]:
while pl.ms.IsInMotion():
    pass
dp=PlotDisplayer()
dp.CreateMaket(mainParam='time',power=pl.data[['time','power']],
                    tension=pl.data[['time','tension','tensionWgl', 'tensionEXPgl']],
                  # motors=pl.data[['time','motorR','motorL','motorM']]
                  # v_and_a=pl.data[['time','vL','vR','aL','aR']]
              )
sl=Slider()
sl.NewSl('v',min=0.1,max=9,value=3,step=0.1)
sl.NewSl('a',min=0.1,max=9,value=7,step=0.1)
sl.NewSl('dvKof', max=3,value=0,step=0.01)
sl.NewSl('dv', max=0.5,value=0.001,step=0.0001)
sl.NewSl('window',min=50, max=10000,value=100, step=50)
sl.NewSl('molibdenH',min=-1.6, max=2.6,value=2.6,step=0.01)
sl.NewSl('T',min=10000, max=200000,value=1700,step=10)
sl.NewBtn('molibden','up molibden','doun molibden')
sl.NewBtn('end','end','end2','end3','vse','bolshe ne jmi','nu pravda ne nado', 'samiy umniy da?')
sl.NewBtn('start dv','start dv','stop dv')
sl.NewBtn('start stabili','start st','stop st')
sl.NewTt('x')
sl.NewTt('L')
sl.NewTt('R')
sl.NewTt('dv')

In [9]:
pl.ms.MotorsControlStart()
t0 = time.time()
t = t0
dp.Show()
sl.Display()
pl.Clear()

with ui_events() as poll:
    while True:
        poll(10)
        t = time.time()
        pl.Read()
        x=pl.data['x'].iloc[-1]
        v=sl.Sl['v']()
        a=sl.Sl['a']()
        T=sl.Sl['T']()
        ww=int(sl.Sl['window']())
        sl.ChangeValueTt('x',x)
        sl.ChangeValueTt('L',pl.ms.L_x(x))
        sl.ChangeValueTt('R',pl.ms.R_x(x))
        if sl.BtnFl['start stabili']:
            dv=pl.obrSvas(T,dv)
        else:
            dv=sl.Sl['dv']()
            if not sl.BtnFl['start dv']:
                dv=0
        sl.ChangeValueTt('dv',dv)        
        if pl.ms.PulMotorsControl(v, a, dv, sl.Sl['molibdenH'](), upFl=sl.BtnFl['molibden'], stFl=sl.BtnFl['end'], dhKof=0.5, ah=9)==-1:
            break
            
        dp.Apdate(for_all=pl.data.iloc[-ww:])
        push_notebook()    
        time.sleep(0.04)
print('ura')
pl.Save()
with ui_events() as poll:
    while sl.BtnFl['end']<3:
        poll(10)
        pl.SetW(2,dw=1,tau=0.5)
print('vse')

xMax= 38.44613234964727 L_x(xMax)= 14.700000000000001   x= 38.186575678600406   L= 15.242577228391255 xEnd= 40.51709108229473 lEnd= 14.700000000000001
error in  motor R  -  position is bad x= 125.88309475609472 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
error in  motor R  -  position is bad x= 132.54178033626803 max position= 99
er

KeyboardInterrupt: 

In [31]:
pl.ms.End() # растановка моторчиков на нули (запускать перед отключением питания)

ending
end


In [32]:
pl.Save() # сохранение данных

data saved
